# This is the Python code Notebook for the Final Capstone Project - The Battle of Neighborhoods as part of IBM Data Science Professional Certificate
## <u>Title: Finding a best location to establish a new Yoga center in Toronto city meeting the client requirements</u>

## <b><u>Business Problem</u></b>

### In this project I would be working on a hypothetical business problem of a client from Ontario Canada and try to solve it by presenting some viable options resulting from the analysis of the available data.
### One local businessman who lives in Ontario Canada (client) wants to invest and open a new Yoga center in Toronto city. Client has a concept in his mind and a fair idea about Toronto city setting, but would like to get some advice in choosing a best location from his perspective to open a new Yoga center and therefore needs help.

### During an initial meeting, client has discussed the business goal that is in his mind and defined some criterion for selecting a spot for Yoga center like listed below.
<ol>
<b><li>It should be located in urban setting like within city limits</li></b>
<b><li>Within the 15 minutes walking distance from the city center</li></b>
<b><li>As close as possible to other popular places in the area and it is as crowded as possible where we can expect heavy foot traffic</li></b>
<b><li>There should be less or no competition</li></b>
<b><li>He needs a place where labor is available easily at reasonable salaries</li></b>
<b><li>In what best location Neighborhood and/ or borough should he open a Yoga center to have the best chance of being successful</li></b>
</ol>

## Import required Python libraries and others

In [1]:
# Import required libraries and others

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

## Data scraped from Toronto Wiki website and added to Data Frame
### Sample data from data frame displayed below

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

reqToronto = requests.get(url)

soupToronto = BeautifulSoup(reqToronto.content,'lxml')

tableToronto = soupToronto.find_all('table')[0]

dfToronto = pd.read_html(str(tableToronto))

Toronto_neighborhood=pd.DataFrame(dfToronto[0])
Toronto_neighborhood.head()
Toronto_neighborhood.shape

(180, 3)

## Data Cleanup

### Deleting the records with Not assigned Borough from DataFrame: Count of rows in dataframe BEFORE and AFTER deleting the Not assigned Borough

In [3]:
# Count of rows in dataframe before deleting Not assigned Borough
print('Count of rows in dataframe BEFORE deleting Not assigned Borough: ', Toronto_neighborhood.shape)

Toronto_neighborhood.drop(Toronto_neighborhood.loc[Toronto_neighborhood['Borough']== "Not assigned"].index, inplace=True)

# Count of rows in dataframe after deleting Not assigned Borough
print('Count of rows in dataframe AFTER deleting Not assigned Borough: ', Toronto_neighborhood.shape)
Toronto_neighborhood.head()

Count of rows in dataframe BEFORE deleting Not assigned Borough:  (180, 3)
Count of rows in dataframe AFTER deleting Not assigned Borough:  (103, 3)


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### If a cell has a Borough but a Not assigned neighborhood, then the neighborhood will be the same as the Borough

In [4]:
Toronto_neighborhood['Neighborhood']=Toronto_neighborhood['Neighborhood'].replace('NaN', Toronto_neighborhood['Borough'])
Toronto_neighborhood.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Combining the duplicate postal codes in to single row.

In [5]:
Toronto_neighborhood.groupby(['Postal code','Borough'],sort=False).agg(lambda x: ','.join(x))
Toronto_neighborhood.reset_index(drop=True, inplace = True)
print("Number of rows of the dataframe: ",Toronto_neighborhood.shape)
Toronto_neighborhood.head()

Number of rows of the dataframe:  (103, 3)


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## All the Python code Functions used in this Notepad are listed below for easy access

In [6]:
# get_category_type function to extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# getNearbyVenues function to extracts the near by venues given a locaiton coordinates and radius around it

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


# return_most_common_venues function returns the most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



## Add Longitute and Lattitude values to all postal codes in the Toronto Neighborhood data frame

In [7]:
# Load coordinate data from CSV file
Toronto_neighborhood = Toronto_neighborhood.rename({'Postal code':'Postal Code'}, axis=1)
lltemp_df = pd.read_csv('http://cocl.us/Geospatial_data')
geome_df = lltemp_df.rename({'Postal code':'PostalCode'}, axis=1)

# Merge coordinates into neighbourhood
Toronto_neighborhood = Toronto_neighborhood.merge(geome_df)
Toronto_neighborhood.shape
Toronto_neighborhood.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# Explore and cluster the neighborhoods in Toronto city to perform data analysis

### Number of Boroughs and Neighborhoods in the data set

In [8]:
print('The dataframe has {} Boroughs and {} Neighborhoods.'.format(
        len(Toronto_neighborhood['Borough'].unique()),
        Toronto_neighborhood.shape[0]
    )
)

The dataframe has 10 Boroughs and 103 Neighborhoods.


## Client request was to focus only within Toronto City to explore a best locaiton to open a new Yoga center
### Considering only the Boroughs in the Toronto City (assumption: take only the Boroghs with word Toronto)

In [9]:
#Focusing only on Boroughs that contain the word Toronto
toronto_dfone = Toronto_neighborhood[Toronto_neighborhood['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_dfone

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.669005,-79.442259


### Find out how many Boroughs and Neighborhoods in the Dataframe

In [10]:
print('The Toronto City Dataframe has {} Boroughs and {} Neighborhoods.'.format(
        len(toronto_dfone['Borough'].unique()),
        toronto_dfone.shape[0]
    )
)

The Toronto City Dataframe has 4 Boroughs and 39 Neighborhoods.


### Obtain the Geographical Coordinates using Geo Locator - latitude and longitude values of Toronto city

In [11]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create map of Toronto city using latitude and longitude values obtained above

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto_dfone['Latitude'], toronto_dfone['Longitude'], toronto_dfone['Borough'], toronto_dfone['Neighborhood']):
    label = '{}, {}'.format(toronto_dfone, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Use the Foursquare API to get locaiton data to explore the neighborhoods

In [13]:
CLIENT_ID = 'LQWEPZR310MZTHC1CKEPP2EPHM2FFONLHK1KWB1GSDN4ZVTS'
CLIENT_SECRET = 'YRZX0HIR31A5M3JTQPDD4GNI2S1ONDSUR1RUEGKW4UGFBMMK'
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LQWEPZR310MZTHC1CKEPP2EPHM2FFONLHK1KWB1GSDN4ZVTS
CLIENT_SECRET:YRZX0HIR31A5M3JTQPDD4GNI2S1ONDSUR1RUEGKW4UGFBMMK


### Use the Python function on each neighborhood to fetch all the venues and create a new dataframe called toronto_venues.

In [14]:
toronto_venues = getNearbyVenues(names=toronto_dfone['Neighborhood'],
                                   latitudes=toronto_dfone['Latitude'],
                                   longitudes=toronto_dfone['Longitude']
                                  )

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Roseda

In [15]:
# Check the size of the resulting dataframe
print(toronto_venues.shape)

(3188, 7)


In [16]:
## Check how many venues were returned for each neighborhood
toronto_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Airport,1,1,1,1,1,1
Airport Lounge,1,1,1,1,1,1
American Restaurant,35,35,35,35,35,35
Amphitheater,2,2,2,2,2,2
Animal Shelter,1,1,1,1,1,1
Antique Shop,2,2,2,2,2,2
Aquarium,3,3,3,3,3,3
Art Gallery,27,27,27,27,27,27


### Find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} unique Venue categories in the Toronto City data.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 unique Venue categories in the Toronto City data.


## Analyze each Neighborhood to find out most common venues

In [18]:
# one hot encoding
torontoarea_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoarea_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torontoarea_onehot.columns[-1]] + list(torontoarea_onehot.columns[:-1])
torontoarea_onehot = torontoarea_onehot[fixed_columns]

torontoarea_onehot.shape
torontoarea_onehot.head()

,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
torontoarea_grouped = torontoarea_onehot.groupby('Neighborhood').mean().reset_index()
torontoarea_grouped

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,0.00,0.000000,0.000000,0.000000,0.00,0

In [20]:
torontoarea_grouped.shape

(39, 275)

### Create the new dataframe and display the top 10 venues for each neighborhood.

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontoarea_grouped['Neighborhood']

for ind in np.arange(torontoarea_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontoarea_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Park,Beer Bar,Art Gallery,Bakery,Creperie
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Restaurant,Bar,Furniture / Home Store,Bakery,Tibetan Restaurant,Gift Shop,Performing Arts Venue,Thrift / Vintage Store
2,Business reply mail Processing CentrE,Coffee Shop,Park,Pizza Place,Brewery,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Beach,Steakhouse,Bistro
3,CN Tower / King and Spadina / Railway Lands / ...,Harbor / Marina,Coffee Shop,Café,Park,Airport,Airport Lounge,Scenic Lookout,Garden,Dance Studio,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Gastropub,Japanese Restaurant,Furniture / Home Store,Juice Bar,Mexican Restaurant,Middle Eastern Restaurant,Diner


## Cluster the Neighborhoods using K-Means Algorithm into 5 Clusters

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = torontoarea_grouped.drop('Neighborhood', 1)
#toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 2, 0, 1, 4, 2, 1, 2, 2], dtype=int32)

### Create a new Dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
torontoarea_merged = toronto_dfone

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torontoarea_merged = torontoarea_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
torontoarea_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Diner,Park,Café,Theater,Restaurant,Italian Restaurant,Bakery,Breakfast Spot
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Park,Gastropub,Ramen Restaurant,Japanese Restaurant,Burger Joint,Pizza Place,Middle Eastern Restaurant,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Japanese Restaurant,Gastropub,Restaurant,Diner,Hotel,Theater,Café,Creperie,Cosmetics Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Hotel,Italian Restaurant,Farmers Market,Japanese Restaurant,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Coffee Shop,Pizza Place,Japanese Restaurant,Breakfast Spot,Beach,Grocery Store,Park,Bar,Bakery


### Create Toronto City map to visualize the resulting clusters

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontoarea_merged['Latitude'], torontoarea_merged['Longitude'], torontoarea_merged['Neighborhood'], torontoarea_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Data Analysis to arrive at a Solution to the given Business Problem

### Examine First Cluster data (sample)

In [25]:
torontoarea_merged.loc[torontoarea_merged['Cluster Labels'] == 0, torontoarea_merged.columns[[1] + list(range(5, torontoarea_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Diner,Park,Café,Theater,Restaurant,Italian Restaurant,Bakery,Breakfast Spot
9,West Toronto,0,Café,Park,Coffee Shop,Bar,Sushi Restaurant,Bakery,Grocery Store,Pharmacy,Italian Restaurant,Gourmet Shop
32,Downtown Toronto,0,Harbor / Marina,Coffee Shop,Café,Park,Airport,Airport Lounge,Scenic Lookout,Garden,Dance Studio,Sculpture Garden
33,Downtown Toronto,0,Grocery Store,Park,Coffee Shop,Breakfast Spot,BBQ Joint,Playground,Convenience Store,Sandwich Place,Candy Store,Metro Station


### Data Cleaning: Create a new data frame with fewer columns for Analysis

In [26]:
torontoarea_analize = torontoarea_merged[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels']]
torontoarea_analize.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2


In [27]:
toronto_venues.shape

(3188, 7)

In [28]:
torontoarea_analize = torontoarea_analize.join(toronto_venues.set_index('Neighborhood'), on='Neighborhood')
torontoarea_analize.shape

(3188, 12)

In [29]:
torontoarea_analize.groupby('Cluster Labels').count()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Cluster Labels,,,,,,,,,,,
0,204,204,204,204,204,204,204,204,204,204,204
1,1034,1034,1034,1034,1034,1034,1034,1034,1034,1034,1034
2,1341,1341,1341,1341,1341,1341,1341,1341,1341,1341,1341
3,9,9,9,9,9,9,9,9,9,9,9
4,600,600,600,600,600,600,600,600,600,600,600


### Create a temp dataframe to capture the total numbers of vanues in each cluster to see the popularity of the areas

In [30]:
temp_df2 = pd.DataFrame(torontoarea_analize['Cluster Labels'].value_counts().reset_index().values, columns=["Cluster Labels", "Total Venues"])
temp_df3 = temp_df2.sort_index(axis = 0, ascending=True)
temp_df3

,Cluster Labels,Total Venues
0,2,1341
1,1,1034
2,4,600
3,0,204
4,3,9


### Find out the number of existing Yoga centers in each cluster and put the data into a temp dataframe to figure out how many Yoga centers are there in the most popular areas in Toronto city

In [31]:
temp_df = torontoarea_analize[torontoarea_analize['Venue Category'].str.contains("Yoga", "yoga")]
temp_df.shape

(27, 12)

In [32]:
temp_df4 = pd.DataFrame(temp_df['Cluster Labels'].value_counts().reset_index().values, columns=["Cluster Labels", "Total Yoga Venues"])
temp_df5 = temp_df4.sort_index(axis = 0, ascending=True)
temp_df5

,Cluster Labels,Total Yoga Venues
0,2,16
1,4,5
2,1,5
3,0,1


### Merge the two temporary dataframes created above to represent the total number Yoga centers verses total number of venues in each cluster

In [33]:
torontoarea_yogadata1 = temp_df3.join(temp_df5.set_index('Cluster Labels'), on='Cluster Labels').fillna(0).astype(int)
torontoarea_yogadata = torontoarea_yogadata1.sort_values(by=['Cluster Labels']).reset_index(drop=True)
torontoarea_yogadata

,Cluster Labels,Total Venues,Total Yoga Venues
0,0,204,1
1,1,1034,5
2,2,1341,16
3,3,9,0
4,4,600,5


### Visually look at the Venues and Yoga centers distribution acroess all the Clusters using bar charts

In [34]:
import matplotlib.pyplot as plt
bar_width = 0.35
opacity = 0.8

labels = torontoarea_yogadata['Cluster Labels']
venues_all = torontoarea_yogadata['Total Venues']
venues_yoga = torontoarea_yogadata['Total Yoga Venues']

x = np.arange(len(labels))  # the label locations
width = 0.5  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, venues_all, width, opacity, label='Total Venues')
rects2 = ax.bar(x + width/2, venues_yoga, width, opacity, label='Total Yoga Venues')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Number of Venues')
ax.set_title('Distribution of Venues by Cluster')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
fig.tight_layout()

# Turn on the grid
plt.minorticks_on()
#plt.grid(which='major', linestyle='-', linewidth='0.2', color='blue')
#plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

plt.show()

<Figure size 640x480 with 1 Axes>

### <u>Findings from the data analysis</u>

In [35]:
for i in range(5):
  print('The Cluster {} has a total of {} Venues including {} Yoga Centers.'.format(torontoarea_yogadata.iat[i,0], torontoarea_yogadata.iat[i,1], torontoarea_yogadata.iat[i,2]))

The Cluster 0 has a total of 204 Venues including 1 Yoga Centers.
The Cluster 1 has a total of 1034 Venues including 5 Yoga Centers.
The Cluster 2 has a total of 1341 Venues including 16 Yoga Centers.
The Cluster 3 has a total of 9 Venues including 0 Yoga Centers.
The Cluster 4 has a total of 600 Venues including 5 Yoga Centers.


In [36]:
torontoarea_final = torontoarea_analize[['Postal Code', 'Borough', 'Neighborhood','Cluster Labels']]
torontoarea_final = torontoarea_final[torontoarea_final['Cluster Labels'] == 0]
#torontoarea_final.shape
ListBorough = torontoarea_final.Borough.unique().tolist()
ListNeighborhood = torontoarea_final.Neighborhood.unique().tolist()

print('Based on the data analysis it is recommended that a new Yoga Center can be established in any of the {} neighborhoods in the {} Borough.'.format(ListNeighborhood, ListBorough))

Based on the data analysis it is recommended that a new Yoga Center can be established in any of the ['Regent Park / Harbourfront', 'Dufferin / Dovercourt Village', 'CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport', 'Rosedale'] neighborhoods in the ['Downtown Toronto', 'West Toronto'] Borough.


#### The Cluster <u>0</u> has a total of <u>900</u> Venues including <u>3</u> Yoga Centers.
#### The Cluster <u>1</u> has a total of <u>8</u> Venues including <u>0</u> Yoga Centers.
#### The Cluster <u>2</u> has a total of <u>1214</u> Venues including <u>16</u> Yoga Centers.
#### The Cluster <u>3</u> has a total of <u>15</u> Venues including <u>0</u> Yoga Centers.
#### The Cluster <u>4</u> has a total of <u>1042</u> Venues including <u>7</u> Yoga Centers.

## <u>Few Insights noted based on the analysis as listed below</u>
<ul>
<b><li>Clusters 0, 2 and 4 are appear to be very busy areas with many existing venues</li></b>
<b><li>Clusters 0, 2 and 4 are more or less equally popular based on the number of Venues that each Cluster has</li></b>
<b><li>Clusters 1 and 3 appear to be less popular considering the number of Venues each has, we can eliminate these two clusters from consideration</li></b>
<b><li>clusters 2 and 4 have more number of Yoga centers established already and the cluster 0 has less numbers of Yoga centers</li></b>
<b><li>Competition in Cluster number 0 is less compared to other two Clusters 2 and 4</li></b>
</ul>

### <span style="color:green">**After careful consideration of the data and the insights after data analysis, Cluster 0 found to be the suitable area for establishing a new Yoga center in Toronto city being the popular area with less competition.**</span>

### <span style="color:green"><u>Finally the Client has been adviced and recommended that a new Yoga Center can be established in any of the following Nieghborhoods in the <i>Downtown</i> Burough:</u>
<ol>
<b><li>Garden District, Ryerson</li></b>
<b><li>St. James Town</li></b>
<b><li>Berczy Park</li></b>
<b><li>Richmond / Adelaide / King</li></b>
<b><li>Harbourfront East / Union Station / Toronto Islands</li></b>
<b><li>Toronto Dominion Centre / Design Exchange</li></b>
<b><li>Commerce Court / Victoria Hotel</li></b>
<b><li>Stn A PO Boxes</li></b>
<b><li>First Canadian Place / Underground city</li></b>
</ol>
</span>


## End of Capstone Project - The Battle of Neighborhoods (weeks 1 and 2): Finding a best location to open a new Yoga center in Toronto city.